<a href="https://colab.research.google.com/github/Sushila-Phogat/CE888/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import pandas as pd
import numpy as np

# Assign spreadsheet filename: file

df1 =pd.read_excel('https://github.com/Sushila-Phogat/CE888/blob/master/jester-data-1.xls?raw=true', header=None)



df1.drop(df1.columns[[0]], axis = 1, inplace = True) 

df1.head(5)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,7.33,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,2.82,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [91]:
print(df1.shape)

(24983, 100)


In [92]:
df1.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
count,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,...,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.00000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000
mean,34.401435,27.094089,33.263717,36.244936,0.427236,19.342119,-0.401803,-0.616110,36.228421,19.520763,14.107285,11.445280,-1.754815,11.038325,-1.682015,-3.072122,-1.103119,-0.619151,0.174010,-0.921997,2.336112,14.812794,19.702896,35.003861,16.831144,6.108754,3.492787,5.393417,3.160797,27.448552,3.465963,3.290261,31.892051,14.294251,3.125464,3.381442,31.184107,9.610589,8.527044,11.138871,...,3.147019,3.156669,19.295580,30.078602,3.261098,2.773196,29.075706,2.804985,2.818099,19.205059,64.955502,65.511747,65.230747,63.738037,64.060745,64.784238,63.813772,63.885116,63.075803,63.645943,63.130073,63.174054,62.956359,62.417244,62.222051,61.607885,62.263703,61.753387,62.469264,60.689216,61.462019,60.27989,60.421701,59.059621,58.749897,58.541104,58.017246,57.061650,55.753357,60.138090
std,46.707059,44.228006,46.747535,48.812967,5.472657,37.860770,5.741616,5.008894,48.228364,38.329975,32.641415,29.938683,5.155992,29.526365,5.355089,5.281486,4.532969,5.039152,5.083095,5.216848,6.856606,34.587240,39.682897,48.637167,37.051743,21.688893,7.074558,19.659326,6.401350,44.864605,12.037673,5.717119,47.440118,34.057662,5.703922,4.991137,47.176688,27.673845,26.397293,30.228748,...,9.383020,6.160750,39.238572,46.233500,10.976394,6.659336,45.963476,6.105714,6.822990,38.955997,47.396153,45.921201,46.644798,48.075806,47.520009,46.243016,47.220253,46.804611,47.675717,47.133626,47.002169,47.332326,46.966353,47.621754,47.593983,47.967071,47.183236,47.224021,46.479714,48.088127,47.335405,47.94644,47.359847,48.196204,48.310728,48.130562,48.164681,48.717455,49.237139,47.913531
min,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,...,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.95000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000
25%,-0.150000,-2.180000,-1.145000,-3.350000,-3.690000,-0.390000,-5.290000,-4.610000,-1.775000,-0.830000,-0.530000,-1.020000,-6.260000,-1.410000,-6.210000,-7.430000,-4.420000,-4.420000,-3.450000,-5.050000,-0.730000,-1.840000,-2.480000,-3.690000,-2.280000,-1.500000,0.730000,-1.410000,0.340000,-2.380000,-0.580000,0.680000,-3.640000,-

In [93]:
arr = df1.values
print(arr.shape)
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

(24983, 100)
1810455 (1810455,)


In [94]:
print(arr)
print(np.where(arr!=99))

[[-7.82  8.79 -9.66 ... 99.   99.   99.  ]
 [ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]
(array([    0,     0,     0, ..., 24982, 24982, 24982]), array([ 0,  1,  2, ..., 69, 70, 86]))


In [0]:

def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [97]:
arr[idx[0][0], idx[1][0]]

8.3

In [98]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
n_latent_factors = 2

user_ratings = df1.values
# Initialise as random values
latent_user_preferences = np.random.random((df1.shape[0], n_latent_factors))
latent_item_features = np.random.random((df1.shape[1], n_latent_factors))

In [101]:
latent_item_features.shape[0]

100

In [102]:
latent_user_preferences.shape[0]

24983

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.001):
  prediction_rating = predict_rating(user_id, item_id)
  err =  prediction_rating - rating
  user_pref_values = latent_user_preferences[user_id][:]
  latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
  latent_item_features[item_id] -= alpha * err * user_pref_values
  return err
    

def sgd(iterations=10):
  for iteration in range(iterations):
    error=[]
    for user_id in range(latent_user_preferences.shape[0]):
      for item_id in range(latent_item_features.shape[0]):
        rating = user_ratings[user_id][item_id]
        if (rating!=99):
          err = train(user_id, item_id, rating)
          error.append(err)
    mse = (np.array(error) ** 2).mean()
    #if the iteration multiple of 10000
    if (iteration % 1) == 0:
      print(mse)

In [104]:
sgd()

24.351121887917394
21.52917068258157
18.728175468465462
17.8150686218577
17.71893439269095
17.74828856902953
17.825218341173724
17.947768757637256
18.14237467101266
18.467937635488113


In [111]:

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-2.829, -3.314, -5.059, ..., -1.486, -6.811, -1.314],
       [ 1.38 ,  1.582,  2.079, ...,  1.106,  2.596,  1.11 ],
       [ 6.546,  7.323,  7.777, ...,  7.291,  8.423,  7.787],
       ...,
       [ 0.457,  0.523,  0.672, ...,  0.384,  0.828,  0.389],
       [-0.792, -0.961, -1.792, ..., -0.049, -2.607,  0.085],
       [ 4.023,  4.474,  4.474, ...,  4.781,  4.605,  5.155]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df1.columns

In [113]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -2.8286436957418144)","(8.79, -3.314077568691325)","(-9.66, -5.059370253081388)","(-8.16, -9.163151779903952)","(-7.52, -3.0612752471547657)","(-8.5, -1.5679171474628064)","(-9.85, -5.832625264718522)","(4.17, -5.0898516477029085)","(-8.98, -8.618525073190442)","(-4.76, -4.629894647259178)","(-8.5, -2.06193354350855)","(-6.75, -1.2870389836140308)","(-7.18, -7.880294219950595)","(8.45, -1.378071752546425)","(-7.18, -8.983472422166402)","(-7.52, -12.128101769332682)","(-7.43, -5.487127733360415)","(-9.81, -7.279249236969483)","(-9.85, -5.4163136785121635)","(-9.85, -8.921674738587134)","(-9.37, 0.870110437969158)","(1.5, -4.40587448902964)","(-4.37, -6.963921018098406)","(-9.81, -11.835517576859635)","(-8.5, -5.08465736624621)","(1.12, -2.0140296065666847)","(7.82, 3.4477658491462675)","(2.86, -2.1941607353002213)","(9.13, 3.7144976922543274)","(-7.43, -8.741966993775813)","(2.14, 1.1317965009969586)","(-4.08, 3.3581565201948536)","(-9.08, -11.230443503378458)","(7.82, -3.268564169613338)","(5.05, 2.9758848548688692)","(4.95, 3.991683818926852)","(-9.17, -10.873635967033582)","(-8.4, -4.719606785626713)","(-8.4, -2.8750823397066534)","(-8.4, -4.650463562260445)",...,"(8.59, 0.7328577356281526)","(3.59, 1.6546837987739862)","(-6.84, -7.828090550532546)","(-9.03, -10.53318551493635)","(2.82, 1.5862456646368028)","(-1.36, 1.1792832053222713)","(-9.08, -11.228011157804044)","(8.3, 1.1205343105318335)","(5.68, 1.6266085498109866)","(-4.81, -5.713265923525979)","(99.0, -8.659827858135888)","(99.0, 1.4036252180606983)","(99.0, -3.2147692894777955)","(99.0, -9.740003455639712)","(99.0, -7.495165326285253)","(99.0, -0.17235101472015213)","(99.0, -5.384091546017624)","(-9.42, -3.396159603215884)","(99.0, -7.036039113408104)","(99.0, -4.382455195511536)","(99.0, -1.3947012078292211)","(-7.72, -4.906505803144726)","(99.0, -1.1324184725839033)","(99.0, -5.183257962883403)","(99.0, -4.785673705984248)","(99.0, -6.328656831377778)","(99.0, -1.6180518410491485)","(99.0, -1.7145036873431965)","(99.0, 2.0474196836298892)","(99.0, -6.319578847442455)","(2.82, -2.234804256518051)","(99.0, -3.9157461857173996)","(99.0, -0.39026135689177066)","(99.0, -5.887403837994616)","(99.0, -4.7008598995524284)","(99.0, -3.9033366650612673)","(-5.63, -3.3680187895903355)","(99.0, -1.4862980923590823)","(99.0, -6.810816491269878)","(99.0, -1.3138223152867674)"
1,"(4.08, 1.3795439296794132)","(-0.29, 1.5820783057757926)","(6.36, 2.0786727883422165)","(4.37, 3.123394310389064)","(-2.38, 1.401725279629751)","(-9.66, 1.241816428286046)","(-0.73, 2.1590410854102595)","(-5.34, 1.7055977264294135)","(8.88, 3.0018350881040776)","(9.22, 2.0011951423695)","(6.75, 1.4348624417028795)","(8.64, 1.2430905797886453)","(4.42, 2.492915203869474)","(7.43, 1.2934284587428426)","(4.56, 2.7420812037585582)","(-0.97, 3.540702109613706)","(4.66, 1.8945506157694716)","(-0.68, 2.475616233674097)","(3.3, 2.1634774703904416)","(-1.21, 3.1118153283256107)","(0.87, 0.6612382562756741)","(8.64, 2.0829666721359943)","(8.35, 2.6867890742796114)","(9.17, 3.8671006147055773)","(0.05, 2.2547986384720065)","(7.57, 1.4700705302633617)","(4.71, -0.15553546189216141)","(0.87, 1.4352326748486377)","(-0.39, -0.24004839734950859)","(6.99, 3.212684121131491)","(6.5, 0.5389325738175487)","(-0.92, -0.12079354363660841)","(7.14, 3.884818696595702)","(9.03, 1.8006505771832098)","(-1.8, 0.0034846908855100463)","(0.73, -0.3715061968611383)","(7.09, 3.646873570259791)","(3.4, 2.161417638828878)","(-0.87, 1.6977149841267023)","(7.91, 2.2307862993408083)",...,"(-6.7, 0.6713396634059933)","(-3.35, 0.40728409284070866)","(-9.03, 3.0829348628535755)","(4.47, 3.6768958379407546)","(4.08, 0.4031165278542823)","(-3.83, 0.5291491176205492)","(8.74, 3.887509350287914)","(1.12, 

In [115]:
predictions.mean(axis = 0)

array([ 0.706,  0.74 ,  0.266, -1.163,  0.53 ,  1.615, -0.302, -0.721,
       -0.93 ,  0.497,  1.616,  1.9  , -1.496,  1.938, -1.961, -3.062,
       -0.635, -0.939,  0.126, -0.951,  2.567,  0.931, -0.081, -1.931,
        0.692,  1.754,  3.053,  1.484,  3.103, -0.513,  2.515,  3.052,
       -1.28 ,  1.346,  2.988,  3.044, -1.533,  0.818,  1.476,  1.065,
       -0.757,  2.237, -1.137, -2.737,  1.007,  1.589,  1.617,  1.802,
        2.835,  3.405, -0.333,  0.151,  3.114,  2.832,  0.186,  2.112,
       -2.473, -3.761, -0.649, -1.088,  2.456,  2.701,  0.07 , -1.115,
        2.622,  2.538, -1.271,  2.506,  2.716,  0.599, -1.014,  2.56 ,
        1.12 , -1.31 , -0.368,  2.127,  0.546,  1.394,  0.112,  1.188,
        1.766,  0.494,  1.978,  0.426,  0.721, -0.323,  1.543,  1.579,
        2.885,  0.171,  1.447,  0.898,  2.096,  0.215,  0.5  ,  0.918,
        1.082,  1.348, -0.161,  1.532])

In [180]:
#Best joke
predictions_mean=[]
predictions_mean.append(predictions.mean(axis = 0))
#predictions_mean[-1].sort_values(ascending=False).head(1)
#predictions.mean(axis = 0)[-1].sort_values(ascending=False).head(1)

print("Best joke is: and its mean value is : ", np.argmax(predictions_mean[-1])+1,np.max(predictions_mean[-1]))

Best joke is: and its mean value is :  50 3.4049566190899037


In [181]:
#worst joke
print("Best joke is: and its mean value is : ", np.argmin(predictions_mean[-1])+1,np.min(predictions_mean[-1]))

Best joke is: and its mean value is :  58 -3.760899103133236
